In [23]:
"""Define the loss function between the y_train_estimated and y_train
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
"""
def criterion(y_estimated, y):

    nllcrit = nn.NLLLoss2d()
    
    y_only_usefull_class = (y != (parameters.number_classes - 1))
    y_only_usefull_class = y_only_usefull_class.unsqueeze(1)
    y_only_useless_class = (y == (parameters.number_classes - 1))

    y_estimated = y_estimated * y_only_usefull_class.float()

    y_estimated[:,parameters.number_classes - 1,:,:] = y_only_useless_class.float()

    return nllcrit(F.log_softmax(y_estimated, dim = 1), y)

In [16]:
"""Define the loss function between the y_train_estimated and y_train and return a vector with the epoch the value
    and if the criterion is used on training or validation set.
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
    (2) = epoch : the actual epoch of the learning
    (3) = set_type : string which is train of validation
"""
def criterion_pd_format(y_estimated, y, epoch, set_type):
    nllcrit = nn.NLLLoss2d()
    y_only_usefull_class = (y != (parameters.number_classes - 1))
    y_only_usefull_class = y_only_usefull_class.unsqueeze(1)
    y_only_useless_class = (y == (parameters.number_classes - 1))

    y_estimated = y_estimated * y_only_usefull_class.float()

    y_estimated[:,parameters.number_classes - 1,:,:] = y_only_useless_class.float()
    loss = nllcrit(F.log_softmax(y_estimated, dim = 1), y)
    return ([loss.data[0], set_type, epoch])

In [20]:
"""
    Test function : IoU
    Return a matrix of confusion in a good format to creat a pandas DataFrame
    (0) = y_estimated : result of train(x) which is the forward action
    (1) = y : Label associated with x
    (2) = epoch : the actual epoch of the learning
    (3) = set_type : string which is train of validation
"""
def IoU_pd_format(y_estimated, y, set_type, epoch):
    
    #We keep only the higest value, which is the prediction
    pred = torch.max(y_estimated, dim=1)[1]

    #Creat the confusion matrix, only the second column will have the value
    confusion_matrix = [[0] * 5 for i in range(parameters.number_classes**2)]
    
    pred = pred.view(-1)
    target = y.view(-1)

    # We will normalise the value in the matrix. We don t want to be influence by the size of the batch
    normalisation_value = y.size()

    # It is also possible to add the size of the image to normalise
    normalisation_value = normalisation_value[0]# * normalisation_value[1] * normalisation_value[2]
    
    # Double loop over the number of classes at each iteration we add the intersection
    # i will be the number of iteration
    i = 0
    for cls1 in range(parameters.number_classes):
        pred_inds = pred == cls1
        
        for cls2 in range(parameters.number_classes):
            
            i = cls1*parameters.number_classes + cls2
            
            target_inds = target == cls2
            # Intersection is the value predicted of class cls2 and are in reality class cls1
            intersection = (pred_inds*target_inds).long().sum().data.cpu()[0]
            
            # Traning or validation set
            confusion_matrix[i][0] = set_type
            # The value normalised
            confusion_matrix[i][1] = intersection/normalisation_value
            # Associated with this value we keep the two classes
            confusion_matrix[i][2] = "class" + str(cls1)
            confusion_matrix[i][3] = "class" + str(cls2)
            # The epoch associated
            confusion_matrix[i][4] = epoch

    return(confusion_matrix)
    